In [1]:
import pandas as pd

In [2]:
# df = pd.read_csv('../Data/cleaned_climate_data.csv')
df = pd.read_csv('../Data/climate_disease_dataset.csv')
df.head()

,year,month,country,region,avg_temp_c,precipitation_mm,air_quality_index,uv_index,malaria_cases,dengue_cases,population_density,healthcare_budget
0,2000,1,Palestinian Territory,Central,28.132468,152.083870,110.487231,12.000000,53,145,113,1068
1,2000,2,Palestinian Territory,Central,30.886500,119.591418,83.467928,12.000000,132,48,113,1068
2,2000,3,Palestinian Territory,Central,31.366433,95.876124,93.095292,12.000000,34,80,113,1068
3,2000,4,Palestinian Territory,Central,28.481870,175.315731,105.530192,9.395894,23,133,113,1068
4,2000,5,Palestinian Territory,Central,26.890370,191.445990,60.205979,9.935726,39,74,113,1068


In [3]:
threshold = 100
df['dengue_outbreak'] = df['dengue_cases'].apply(lambda x: 1 if x >= threshold else 0)
# df = df.drop(columns=['date','district','province'])
df = df.drop(columns=['country','region','dengue_cases','year','month'])

In [4]:
X = df.iloc[:,0:12]
X

,avg_temp_c,precipitation_mm,air_quality_index,uv_index,malaria_cases,population_density,healthcare_budget,dengue_outbreak
0,28.132468,152.083870,110.487231,12.000000,53,113,1068,1
1,30.886500,119.591418,83.467928,12.000000,132,113,1068,0
2,31.366433,95.876124,93.095292,12.000000,34,113,1068,0
3,28.481870,175.315731,105.530192,9.395894,23,113,1068,1
4,26.890370,191.445990,60.205979,9.935726,39,113,1068,0
...,...,...,...,...,...,...,...,...
34555,13.040751,369.269763,70.218442,12.000000,149,52,399,0
34556,9.811456,271.028130,74.749481,8.432640,56,52,399,0
34557,12.308443,228.392496,91.096407,8.034754,104,52,399,0
34558,20.197689,196.673067,128.859240,12.000000,142,52,399,0


In [5]:
# y = df.iloc[:,7]
y = df['dengue_outbreak']
y

0        1
1        0
2        0
3        1
4        0
        ..
34555    0
34556    0
34557    0
34558    0
34559    1
Name: dengue_outbreak, Length: 34560, dtype: int64

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import RandomForestRegressor

rf = RandomForestClassifier(n_estimators=100, random_state=42)
# rf = RandomForestRegressor()

rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

rf.score(X_test, y_test)

1.0